In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

In [66]:
# Cargo el data frame de estaciones
estaciones = pd.read_csv('../data/station.csv')
estaciones = estaciones.rename(columns={'id':'station_id'})

# Cada ciudad esta  vinculado a un zip code
zip_codes = {
    'Mountain View': 94041,
    'Redwood City': 94063,
    'San Francisco': 94107,
    'Palo Alto': 94301,
    'San Jose':95113
}

# Creo una nueva columna con zip_code de cada estacion
estaciones['zip_code'] = estaciones.city.apply(lambda c: zip_codes[c])

In [78]:
# Agrego tres columnas.
estaciones["max_dist"] = 0
estaciones["min_dist"] = 0
estaciones["mean_dist"] = 0

In [7]:
# Vamos a hacer bardo con las fechas. Necesitamos trips. Queremos agregar una columna donde se redondea 
# a 15 minutos la fecha de llegada.

trips = pd.read_csv("../data/trip_train.csv", parse_dates=["start_date", "end_date"],\
                    infer_datetime_format=True)

In [13]:
trips["round_start"] = trips.start_date.dt.round("30min")
trips["round_end"] = trips.end_date.dt.round("30min")

In [47]:
# Ahora queremos los 10 destinos más populares a las 8.
t = pd.datetime(2017,1,1,8,0).time()
estaciones = trips[trips.round_start.dt.time == t].end_station_id

In [48]:
estaciones.value_counts().head(10)

70    2784
65    1982
77    1737
61    1540
51    1407
60    1391
74    1244
55    1181
69    1148
48    1142
Name: end_station_id, dtype: int64

In [82]:
# Listo. Podemos hacer eso para cada hora y guardar todo en un arreglo de 48 lugares.
top10 = 48*[0]

horas = 0
minutos = 0
for mediaHora in range(48):
    t = pd.datetime(2017,1,1,horas,minutos).time()
    # Saco el top10
    top10[mediaHora] = trips[trips.round_start.dt.time == t].end_station_id.head(10)
    if minutos == 30:
        horas+=1
        minutos = 0
    else:
        minutos = 30

In [84]:
# Ahora que tengo el top10 para cada media hora con su id debería buscar la latitud y longitud de cada uno.
for i in range(len(top10)):
    top10[i] = pd.DataFrame({"station_id": top10[i].values}).merge(estaciones)[["lat","long"]]

In [127]:
# Ponele que funciona. Qué hago ahora que tengo ese dataframe? 
# Para cada viaje ver que la distancia entre la salida y los top10 que le corresponden a esa hora. 
# De eso max, min y mean.

horas = 0
minutos = 0
for top10i in top10:
    t = pd.datetime(2017,1,1,horas,minutos).time()
    # Con cuáles tengo que comparar?
    idx_hora = trips.round_start.dt.time == t
    # En top10 i tenemos las distintas estaciones. Podemos hacer un for en esas y fue.
    for st_id in estaciones.station_id:
        idx_id = estaciones.station_id == st_id
        reg = estaciones[idx_id]
        lat_i = reg.lat.values[0]
        long_i = reg.long.values[0]
        distancias_i = np.sqrt((lat_i - top10i.lat)**2 + (long_i - top10i.long)**2)
        distancias_i = distancias_i[distancias_i > 0]
        
        estaciones.loc[idx_id, "max_dist_pop_" + str(t)] = distancias_i.max()
        estaciones.loc[idx_id, "min_dist_pop_" + str(t)] = distancias_i.min()
        estaciones.loc[idx_id, "mean_dist_pop_" + str(t)] = distancias_i.mean()
    
    if minutos == 30:
        horas+=1
        minutos = 0
    else:
        minutos = 30
# POSTA: es igual que lo de abajo, pero con un for exterior por hora y filtrando por la misma.

In [110]:
# Calculo distancias minima media y maxima de cada una al resto.
for st_id in estaciones.station_id:
    reg = estaciones[estaciones.station_id == st_id]
    lat_i = reg.lat.values[0]
    long_i = reg.long.values[0]
    distancias_i = np.sqrt((lat_i - estaciones.lat)**2 + (long_i - estaciones.long)**2)
    distancias_i = distancias_i[distancias_i > 0]
    estaciones.loc[estaciones.station_id == st_id, "max_dist"] = distancias_i.max()
    estaciones.loc[estaciones.station_id == st_id, "min_dist"] = distancias_i.min()
    estaciones.loc[estaciones.station_id == st_id, "mean_dist"] = distancias_i.mean()

In [145]:
# Agrego el promedio de duraciones desde cada estación.
promedios = trips[["start_station_id", "duration"]].groupby("start_station_id").mean().reset_index()
promedios = promedios.rename(columns={"start_station_id":"station_id", "duration": "dur_prom"})
estaciones = estaciones.merge(promedios)

In [157]:
def outliers(viajes):
    #Sacamos los outliers
    if('duration' in viajes.columns):
        viajes = viajes.loc[viajes.duration < viajes.duration.quantile(0.99),:]
#         viajes = viajes.loc[viajes.duration > viajes.duration.quantile(0.01),:]
        return viajes
    else:
        return viajes

In [153]:
def add_weather(viajes, soloLluvia = True):
    # Merge provisorio con stations para obtener el zip_code
    viajes_merged  = pd.merge(viajes, estaciones, left_on='start_station_id', right_on='station_id')\
    .drop(['station_id', 'name', 'installation_date', 'lat', 'long','city','dock_count'], axis=1)
    
    weather = pd.read_csv('../data/weather.csv',\
                      parse_dates=["date"] ,\
                      infer_datetime_format=True)
    
    # Events. Ver si da mejor resultado con o sin unir los dos tipos de lluvia.
    if soloLluvia:
        weather.loc[weather.events.isin(['Rain','Rain-Thunderstorm']), 'events'] = 'rain'
        weather.loc[weather.events.isnull(), 'events'] = 'normal'
    
    weather.precipitation_inches = pd.to_numeric(weather.precipitation_inches, errors='coerse')
    weather.precipitation_inches.fillna(0,inplace=True)
    weather['date'] = weather['date'].apply(lambda x: x.date())
    weather['zip_code'] = weather['zip_code'].astype('int64')
    viajes_merged["date"] = viajes_merged["start_date"].apply(lambda x: x.date())
    return weather.merge(viajes_merged, on=['date', 'zip_code']).drop("zip_code",axis=1)

In [154]:
def separarFecha(data):
    data['year'] = data.start_date.dt.year
    data['month'] = data.start_date.dt.month
    data['day'] = data.start_date.dt.day
    data['week_end'] = data.start_date.dt.dayofweek > 5
    data['start_hour'] = data.start_date.dt.hour
    data['start_minutes']= data.start_date.dt.minute
    #Agrego una dimension con la estacion del año
    data['season'] = data.start_date.apply(lambda dt: (dt.month%12 + 3)//3)
    return data.drop("start_date", axis=1)

In [155]:
def categorias(data):
    for col in ['events', 'subscription_type', 'start_station_id', 'season']:
        data[col] = data[col].astype('category')

    data = data.set_index('id')
    data = data.drop(['date'], axis=1)
    # One-hot-encoding: ver la alternativa de sklearn.
    for col in data.columns:
        if 'category' == str(data[col].dtypes):
            data = pd.concat([data, pd.get_dummies(data[col], prefix=col)], axis=1).drop(col, axis=1)
    return data

In [156]:
def preproc(archViajes):
    data = pd.read_csv(archViajes,\
                    parse_dates=["start_date", "end_date"],\
                    infer_datetime_format=True)
    data.drop(['end_station_name', 'end_station_id', 'end_date', 'start_station_name','bike_id', 'zip_code'], \
                axis=1, inplace=True)
    data = outliers(data)
    data = add_weather(data)
    data = separarFecha(data)
    data = categorias(data)
    data.to_csv(archViajes.replace("trip","data"))

In [159]:
arch_train = "../data/trip_train.csv"
arch_test = "../data/trip_test.csv"

In [ ]:
preproc(arch_train)
preproc(arch_test)